In [20]:
from google.colab import drive

drive.mount('/content/drive/')

%cd "/content/drive/MyDrive/CONSEGNA_ML/"


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/.shortcut-targets-by-id/1vnO-rlfCpoeqYPLaBVG66OUTYbxtehIA/CONSEGNA_ML


#IMPLEMENT HERE YOUR CODE

In [21]:
from torchvision import transforms
import torch

##IMPLEMENT HERE THE FUNCTION TO LOAD YOUR MODEL
For example, here we use a single convolutional layer.

In [22]:
from models import DeepLabV3, CNN_7_Layers, DeepLabV3Lite

def load_model():
  model_instance = DeepLabV3Lite()
  # checkpoint = torch.load('checkpoint_epoch_50.pth')
  checkpoint = torch.load('checkpoint_epoch_15.pth', map_location=torch.device('cpu'))
  model_instance.load_state_dict(checkpoint['model_state_dict'])
  model_instance.eval()
  return model_instance

##IMPLEMENT HERE YOUR PREDICT FUNCTION

In [23]:
from utils import convert_onehotencoding_to_rgb

def predict(model, X):
    # accept (3, H, W) or (1, 3, H, W), values in [0, 1]
    # returns an image (3,H,W) that went through the model (with pixel segmentation)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    # ensure dimension of the model (1, 3, H, W)
    transform = transforms.Compose([
        transforms.ToTensor(),  # Converts to (3, H, W), values in [0,1]
    ])

    image_tensor = transform(X).unsqueeze(0).to(device)  # (1, 3, H, W)

    with torch.no_grad():
        output = model(image_tensor)           # (1, 9, H, W)
        predicted = output.argmax(dim=1)       # (1, H, W)
        predicted = predicted.squeeze(0).cpu().numpy()  # (H, W) - class indices

    predicted_rgb = convert_onehotencoding_to_rgb(predicted)

    return predicted_rgb

#DO NOT MODIFY THE CODE BELOW!

This is exactly the code we run for the final test.

After implementing the previous functions, run this code to verify that it works

In [24]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Metrics
def compute_iou(mask1, mask2, label):
  intersection = np.sum((mask1 == label) & (mask2 == label))
  union = np.sum((mask1 == label) | (mask2 == label))
  if union == 0:
    return np.nan
  return intersection / union
def compute_all_iou(mask1, mask2, num_labels=8):
  iou_scores = np.zeros((num_labels))
  for label in range(num_labels):
    iou = compute_iou(mask1, mask2, label+1) # we skip the background label
    iou_scores[label] = iou
  return iou_scores


# Run YOUR LOAD_MODEL FUNCTION
model = load_model()

# Main loop
test_dir = "./train"
# test_dir = '../../datasets/esame_ml/train' # we will change this path with that of the private test set directory
samples = os.listdir(test_dir)
IOUs = np.zeros((len(samples), 8))
verbose = False

for i, subdir in tqdm(enumerate(samples), desc="Processing samples"):
    subdir_path = os.path.join(test_dir, subdir)

    if os.path.isdir(subdir_path):
        # Get the data paths
        rgb_path = os.path.join(subdir_path, 'rgb.jpg')
        labels_path = os.path.join(subdir_path, 'labels.png')

        if os.path.exists(rgb_path) and os.path.exists(labels_path):
            if verbose:
                print(f"Processing subdirectory: {subdir}")

            try:  # ATTENTION: any error occurring in this try-catch means that the corresponding IOUs are evaluated as ZERO

                # Open images
                rgb_image = Image.open(rgb_path)
                rgb_array = np.asarray(rgb_image).copy()
                labels_image = Image.open(labels_path).copy()
                labels_array = np.asarray(labels_image)
                if verbose:
                    print(f"  Loaded {rgb_path} and {labels_path}")

                # Run YOUR PREDICT FUNCTION
                predicted_labels_array = predict(model, rgb_array)

                # Evaluate the IOU metric
                IOUs[i,:] = compute_all_iou(labels_array, predicted_labels_array)

                if verbose:
                    labels_vals = np.unique(np.asarray(labels_image))
                    print(f"  Unique labels values: {labels_vals}")
                    predicted_labels_vals = np.unique(np.asarray(predicted_labels_array))
                    print(f"  Unique predicted labels values: {predicted_labels_vals}")

                    plt.subplot(1, 3, 1)
                    plt.imshow(rgb_image)
                    plt.subplot(1, 3, 2)
                    plt.imshow(labels_image)
                    plt.subplot(1, 3, 3)
                    plt.imshow(predicted_labels_array)
                    plt.show()

                rgb_image.close()
                labels_image.close()

            except FileNotFoundError:
                print(f"  Error: Could not find image files in {subdir_path}")
            except Exception as e:
                print(f"  Error processing images in {subdir_path}: {e}")
        else:
            print(f"  Skipping subdirectory {subdir}: rgb.jpg or labels.png not found.")

score = np.nanmean(IOUs)
print(f"\nFinal competition score: {score}")

Processing samples: 2it [00:01,  1.55it/s]

  Error processing images in ./train/0287: Expected shape (9, M, N)


Processing samples: 3it [00:02,  1.27it/s]

  Error processing images in ./train/1171: Expected shape (9, M, N)


Processing samples: 4it [00:02,  1.32it/s]

...

  Error processing images in ./train/1269: Expected shape (9, M, N)


Processing samples: 125it [02:05,  1.02s/it]

  Error processing images in ./train/0198: Expected shape (9, M, N)


Processing samples: 125it [02:05,  1.01s/it]


KeyboardInterrupt: 

More information useful for

In [ ]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
print(f"All IOUs:\n{IOUs}")
print("Average IOUs for each:")
print(f"- class: {np.nanmean(IOUs, 0)}")
print(f"- image: {np.nanmean(IOUs, 1)}")